In [ ]:
function dosvdtrunc(AA,m)		# AA a matrix;  keep at most m states
    (u,d,v) = svd(AA)
    prob = dot(d,d)		# total probability
    mm = min(m,length(d))	# number of states to keep
    d = d[1:mm]			# middle matrix in vector form
    trunc = prob - dot(d,d)
    U = u[:,1:mm]
    V = v[:,1:mm]'
    (U,d,V,trunc)		# AA == U * diagm(d) * V	with error trunc
end

function dosvdleftright(AA,m,toright)
    (U,d,V,trunc) = dosvdtrunc(AA,m)
    if toright
	V = diagm(d) * V
    else
	U = U * diagm(d)
    end
    (U,V,trunc)
end

function dosvd4(AA,m,toright)	# AA is ia * 2 * 2 * ib;  svd down the middle;  return two parts
    ia = size(AA,1)
    ib = size(AA,4)
    AA = reshape(AA,ia*2,2*ib)
    (U,V,trunc) = dosvdleftright(AA,m,toright)
    mm = size(U,2)
    U = reshape(U,ia,2,mm)
    V = reshape(V,mm,2,ib)
    (U,V,trunc)
end

using TensorOperations

function JK(a,b)	# Julia kron,  ordered for julia arrays; returns matrix
    #     return kron(b,a)
    (a1,a2) = size(a)
    (b1,b2) = size(b)
    reshape(Float64[a[i,ip] * b[j,jp] for i=1:a1, j=1:b1, ip=1:a2, jp=1:b2],a1*b1,a2*b2)
end

function JK4(a,b)	# Julia kron,  ordered for julia arrays, return expanded into 4 indices
    (a1,a2) = size(a)
    (b1,b2) = size(b)
    Float64[a[i,ip] * b[j,jp] for i=1:a1, j=1:b1, ip=1:a2, jp=1:b2]
end

sz = Float64[0.5 0; 0 -0.5]
sp = Float64[0 1; 0 0]
sm = sp'
Htwosite = reshape(JK(sz,sz) + 0.5 * JK(sp,sm) + 0.5 * JK(sm,sp),2,2,2,2)
# order for Htwosite is s1, s2, s1p, s2p

n = 28		# exact n=28 energy is -12.2254405486
#  Make initial product state in up down up down up down pattern (Neel state)
# Make first tensor a 1 x 2 x m tensor; and last is m x 2 x 1  (rather than vectors)
A = [zeros(1,2,1) for i=1:n]
for i=1:n
    A[i][1,iseven(i) ? 2 : 1,1] = 1.0	
end

HLR = [zeros(1,1) for i=1:n]	# Initialize to avoid errors on first sweep
m = 3
for swp = 0:1
    m = round(Int64,1.3*m)
    for ii=-n+1:n-1		# if negative, going right to left
#         @show HLR
        ii == 0 && continue
        i = abs(ii)
        toright = ii > 0

        println("\n sweep, i, dir, m = $swp, $i, ",toright ? "to right" : "to left"," $m")

        dleft = size(A[i],1)
        alpha = dleft * 2
        dright = size(A[i+1],3)
        beta = 2 * dright
        onesite = eye(2)

        HL = zeros(dleft,2,dleft,2)
        HR = zeros(2,dright,2,dright)
        if i > 1
            Aim1 = A[i-1] # Ai  minus 1
            @tensor begin
            HL[a,si,ap,sip] := Htwosite[sim1,si,sim1p,sip] * Aim1[b,sim1,a] * Aim1[b,sim1p,ap]
            end
            i > 2 && ( HL += JK4(HLR[i-1],onesite) )
        end
        HL = reshape(HL,alpha,alpha)
        if i < n-1
            Ai2 = A[i+2]
            @tensor begin
            HR[si1,b,si1p,bp] := Htwosite[si1,si2,si1p,si2p] * Ai2[b,si2,a] * Ai2[bp,si2p,a]
            end
            i < n-2 && (HR += JK4(onesite,HLR[i+2]) )
        end
        HR = reshape(HR,beta,beta)

        Oleft =  Any[JK(eye(dleft),sz), 0.5*JK(eye(dleft),sp), 0.5*JK(eye(dleft),sm)]
        Oright = Any[JK(sz,eye(dright)),JK(sm,eye(dright)),JK(sp,eye(dright))]

        Ai = A[i]
        Ai1 = A[i+1]
        @tensor begin
            AA[a,b,d,e] := Ai[a,b,c] * Ai1[c,d,e]
        end

    #  Inefficient implementation:  m^4   Ham construction
        Ham = zeros(alpha*beta,alpha*beta)
        for j=1:length(Oleft)
            ### why reshape again?
            Ham += JK(reshape(Oleft[j],alpha,alpha),reshape(Oright[j],beta,beta))  
        end
        if i > 1
            Ham += JK(HL,eye(beta))
        end
        if i < n-1
            Ham += JK(eye(alpha),HR)
        end
        ## What's the extra reshapes for? JK reshape everything
        bigH = reshape(Ham,alpha*beta,alpha*beta)
        bigH = 0.5 * (bigH + bigH')
        #why do you choose AA for initial vector? beacuse it was ground state in the last sweep
        evn = eigs(bigH;nev=1, which=:SR,ritzvec=true,v0=reshape(AA,alpha*beta))
        @show evn[1]
        @show size(evn[2])
        gr = evn[2][:,1]

        AA = reshape(gr,dleft,2,2,dright)

        (A[i],A[i+1],trunc) = dosvd4(AA,m,toright)
        @show trunc
        if toright 
            if 1 < i < n-1
            (i1,i2,i3) = size(A[i])
            Ai2 = reshape(A[i],i1*i2,i3)
            @tensor begin
                hlri[b,bp] := HL[a,ap] * Ai2[a,b] * Ai2[ap,bp]
            end
            HLR[i] = hlri
            end
        else
            if 1 < i < n-1
            (i1,i2,i3) = size(A[i+1])
            Ai12 = reshape(A[i+1],i1,i2*i3)
            @tensor begin
                hlri1[a,ap] := HR[b,bp] * Ai12[a,b] * Ai12[ap,bp]
            end
            HLR[i+1] = hlri1
            end
        end
    end
end



In [1]:
a1, a2, b1, b2 = 2, 2,3,3

A = reshape(["a[$i,$(ip)]" for  i=1:a1, ip=1:a2],a1, a2)
@show A
B = reshape(["b[$i,$(ip)]" for  i=1:b1, ip=1:b2],b1, b2)
@show B
@show B[2,3]
@show k = kron(B,A)
# jk = reshape(["a[$i,$(ip)] * b[$j,$(jp)]"
#     for i=1:a1, j=1:b1, ip=1:a2, jp=1:b2],a1*b1,a2*b2)
jk = reshape(["b[$j,$(jp)]a[$i,$(ip)] "
    for i=1:a1, j=1:b1, ip=1:a2, jp=1:b2],a1*b1,a2*b2)

@show jk[2,5]
@show k[2,5]

# @show ["b[$i,$(ip)]" for  i=1:b1, ip=1:b2]
# @show reshape([i * n for i=1:6, n=1:2],3,4)
# reshape(["A", "b"],2,1)

A = Union{ASCIIString,UTF8String}["a[1,1]" "a[1,2]"
                              "a[2,1]" "a[2,2]"]
B = Union{ASCIIString,UTF8String}["b[1,1]" "b[1,2]" "b[1,3]"
                              "b[2,1]" "b[2,2]" "b[2,3]"
                              "b[3,1]" "b[3,2]" "b[3,3]"]
B[2,3] = "b[2,3]"
k = kron(B,A) = Union{ASCIIString,UTF8String}["b[1,1]a[1,1]" "b[1,1]a[1,2]" "b[1,2]a[1,1]" "b[1,2]a[1,2]" "b[1,3]a[1,1]" "b[1,3]a[1,2]"
                              "b[1,1]a[2,1]" "b[1,1]a[2,2]" "b[1,2]a[2,1]" "b[1,2]a[2,2]" "b[1,3]a[2,1]" "b[1,3]a[2,2]"
                              "b[2,1]a[1,1]" "b[2,1]a[1,2]" "b[2,2]a[1,1]" "b[2,2]a[1,2]" "b[2,3]a[1,1]" "b[2,3]a[1,2]"
                              "b[2,1]a[2,1]" "b[2,1]a[2,2]" "b[2,2]a[2,1]" "b[2,2]a[2,2]" "b[2,3]a[2,1]" "b[2,3]a[2,2]"
                              "b[3,1]a[1,1]" "b[3,1]a[1,2]" "b[3,2]a[1,1]" "b[3,2]a[1,2]" "b[3,3]a[1,1]" "b[3,3]a[1,2]"
                              "b[3,1]a[2,1]" "b[3,1]a[2,2]" "b[3,2]a[2,1]" "b[3,2]a[2,2]"

"b[1,3]a[2,1]"

In [ ]:
@show a = reshape(1:4 , 2,2)
@show b = reshape(11:14, 2,2)
# b =[1 2; 1 4]
# reshape(a,9)
@show kron(a,b)
@show reshape(kron(a,b),16)
@show kron(b,a)

In [ ]:
# test = [zeros(1,10)...]
# @show test
# test = zeros(1,10)
# @show test
test = [zeros(1,1) for i=1:10]
@show test
test[1] = ones(2,2)
# test = [zeros(1,1)]
# test[1] = ones(2)
# @show test

# test = zeros(1,10)
# @show test
# test[1] = ones(2)
# @show [i for i=0:5]

In [7]:
function JK(a,b)	# Julia kron,  ordered for julia arrays; returns matrix
    #     return kron(b,a)
    (a1,a2) = size(a)
    (b1,b2) = size(b)
    reshape(Float64[a[i,ip] * b[j,jp] for i=1:a1, j=1:b1, ip=1:a2, jp=1:b2],a1*b1,a2*b2)
end

a = rand(10,22)
b = rand(15, 10)
@time M2 = kron(b,a)
@time M1 = JK(a,b)
M1==M2

  0.000082 seconds (6 allocations: 258.031 KB)
  0.025545 seconds (23.91 k allocations: 1.237 MB)


true

In [8]:
function JK4(a,b)	# Julia kron,  ordered for julia arrays, return expanded into 4 indices
    (a1,a2) = size(a)
    (b1,b2) = size(b)
    Float64[a[i,ip] * b[j,jp] for i=1:a1, j=1:b1, ip=1:a2, jp=1:b2]
end

size(JK4(a,b))


(10,15,22,10)

In [15]:
sz = sparse(Float64[0.5 0; 0 -0.5])
sp = sparse(Float64[0 1; 0 0])
sm = sparse(sp')
temp =kron(sz,sz) + 0.5 * kron(sp,sm) + 0.5 * kron(sm,sp)
Htwosite = reshape(temp,2,2,2,2)

LoadError: LoadError: MethodError: `spzeros` has no method matching spzeros(::Type{Float64}, ::Int64, ::Int64, ::Int64, ::Int64)
Closest candidates are:
  spzeros(::Type{T}, ::Integer, ::Integer)
  spzeros(::Type{T}, !Matched::Type{T}, ::Integer, ::Integer)
  spzeros(!Matched::Integer, ::Integer)
while loading In[15], in expression starting on line 5